In [1]:
import psycopg2
import pandas as pd
import json
import datetime

In [2]:
aujourd= datetime.datetime.today()
aujourd
print(f"{aujourd:%d %m  %Y}")

13 07  2022


In [2]:
def get_index(nom, nombre):
    data = {"a": [2, 1, 10], "b": [5,3,5] }
    l = data[nom]
    return l[nombre]

In [3]:
get_index(nom="b", nombre=1)

3

- Afficher pour chaque annexe, la liste des champs (./app/config.py)
- Pour chaque annexe, créer la requête pour récupérer les données concernant l'annexe de manière variabilisé
- Créer une fonction qui prend en argument une connexion à la base de données, l'année, le type d'annexe et la nature de décision. Cette fonction fait une requête à la base de donnée avec ces arguments. En retour elle renvoie un dataframe pandas 
- Créer une fonction qui "nettoie" le jeu de données (filtre des colonnes, renommage de champs et extraction du json de la colonne json_annexe) qui prend en entrée un dataframe et génère un dataframe.
- Créer une fonction qui génère un CSV à partir d'un dataframe et qui prends en entrée le dataframe ainsi qu'un nom de fichier
- Mettre en place les boucles nécessaires permettant de requêter la base de données pour toutes les annexes pour l'année 2020 et la nature de décision 09.

In [21]:
import app.backend.config as conf
data = conf.CHAMPS_ANNEXES

In [6]:
def annexe(nom):
    #l=conf.CHAMPS_ANNEXES
    return data[nom]
annexe(nom='DATA_EMPRUNT')

['AnEncaisse',
 'AnnuitNetDette',
 'Champ_Editeur',
 'CodArticle',
 'CodNatEmpr',
 'CodPeriodRemb',
 'CodPeriodRembDtVote',
 'CodPeriodRembReneg',
 'CodProfilAmort',
 'CodProfilAmortDtVote',
 'CodProfilAmortReneg',
 'CodTypEmpr',
 'CodTypEmprGaranti',
 'CodTypPreteur',
 'CodTypTxCouv',
 'CodTypTxReneg',
 'CodTyptxDtVote',
 'CodTyptxInit',
 'CoutSortie',
 'Couverture',
 'Dt1RembInit',
 'DtDebCouv',
 'DtEmission',
 'DtFinContr',
 'DtFinCouv',
 'DtPeriodeBonif',
 'DtRegltCouv',
 'DtReneg',
 'DtSignInit',
 'DureeAnn',
 'DureeContratInit',
 'DureeContratReneg',
 'DureeRest',
 'DureeRestInit',
 'DureeRestReneg',
 'IndSousJacent',
 'IndSousJacentApresCouv',
 'IndSousJacentAvantCouv',
 'IndSousJacentDtVote',
 'IndexTxVariDtVote',
 'IndexTxVariInit',
 'IndexTxVariReneg',
 'IndiceCouv',
 'IndiceEmpr',
 'IndiceEmprDtVote',
 'LibCpte',
 'LibOrgCoContr',
 'LibOrgaPreteur',
 'MPrimeRecueCouv',
 'MtCRDCouvert',
 'MtCRDRefin',
 'MtCapitalExer',
 'MtCapitalReamenage',
 'MtCapitalRestDu_01_01',
 'MtCapi

In [12]:
conn = psycopg2.connect(database ="actes_budgetaire", user = "postgres",
                        password = "vGXGLqTY4EF7DCiozYoKlqRRUaC7ECd", host = "ab-postgres",
                        port = "5432")


def get_data(connexion,annexe, year):
    query= """SELECT * from annexe 
    JOIN documentbudgetaire 
        ON documentbudgetaire.id = annexe.fk_id_document_budgetaire 
    JOIN collectivite 
        ON collectivite.siret_coll = documentbudgetaire.fk_siret_collectivite 
    WHERE annexe.type_annexe = 'DATA_EMPRUNT'
    AND documentbudgetaire.exercice = '2020'
    AND documentbudgetaire.nature_dec = '09' 
        LIMIT 100 """
    dat = pd.read_sql_query(query, conn)
    conn.close()
    return dat


In [13]:
get_data(connexion= conn, annexe= 'DATA_EMPRUNT', year='2020')

,id,type_annexe,json_annexe,fk_id_document_budgetaire,siret_etablissement,list_annexes,id,libelle,code_insee,nomenclature,...,num_dec,nature_vote,type_budget,id_etabl_princ,json_budget,fk_siret_collectivite,siret_coll,libelle_collectivite,nature_collectivite,departement
0,50797,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",8911,20005611700019,"[DATA_EMPRUNT, DATA_CONCOURS, DATA_FORMATION, ...",8911,CHAMBOIS,None,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""165"", ""ArtSpe"": ""false"", ""CodRD"":...",20005611700019,20005611700019,CHAMBOIS,Commune,27
1,50807,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",8912,21670145800012,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",8912,COMMUNE DE FRIEDOLSHEIM,None,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""60611"", ""LibCpte"": ""Eau et assain...",21670145800012,21670145800012,COMMUNE DE - FRIEDOLSHEIM,Commune,None
2,50843,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",8917,21700329200018,"[DATA_EMPRUNT, DATA_TRESORERIE, DATA_FISCALITE...",8917,BUDGET COMMUNAL,70329,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""001"", ""ContNat"": ""001"", ""ArtSpe"":...",21700329200018,21700329200018,COMMUNE DE MALVILLERS,COMMUNE,70
3,50852,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",8918,21310135500013,"[DATA_EMPRUNT, DATA_TIERS, DATA_CONCOURS, DATA...",8918,Budget Communal,31135,M14-M14_COM_SUP3500,...,None,3,P,None,"[{""Nature"": ""28184"", ""Fonction"": ""112"", ""ContN...",21310135500013,21310135500013,Commune de CAZERES,Commune,31
4,50870,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",8919,21910671300056,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",8919,Principal,0091671,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""020"", ""ArtSpe"": ""false"", ""CodRD"":...",21910671300056,21910671300056,BUDGET COMMUNAL VILLENEUVE SUR AUVERS,BUDGET COMMUNAL VILLENEUVE SUR AUVERS,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,54185,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",9500,21820166300018,"[DATA_EMPRUNT, DATA_FISCALITE, DATA_CONSOLIDAT...",9500,BUDGET COMMUNAL M14,82008166,M14-M14_COM_INF500,...,None,1,P,None,"[{""Nature"": ""60631"", ""ContNat"": ""011"", ""ArtSpe...",21820166300018,21820166300018,COMMUNE DE ST. MICHEL,COMMUNE,82
96,54323,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",9521,20004172100073,"[DATA_EMPRUNT, DATA_CONSOLIDATION, DATA_PATRIM...",9521,CCHC BASE TOURISTIQUE DE PASSAVANT,70171,M4-M4,...,None,1,A,20004172100099,"[{""Nature"": ""2135"", ""ContNat"": ""21"", ""ArtSpe"":...",20004172100073,20004172100073,COMMUNAUTE DE COMMUNE DE LA HAUTE COMTE,Communauté de commune à FPU,70
97,54330,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""97319.93""}]",9522,21310017500016,"[DATA_EMPRUNT, DATA_TIERS, DATA_FISCALITE, DAT...",9522,Budget COMMUNAL,31017,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""6553"", ""ContNat"": ""65"", ""ArtSpe"":...",21310017500016,21310017500016,Commune d ARLOS,COMMUNE,31
98,54340,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""P"", ""...",9524,21600007500012,"[DATA_EMPRUNT, DATA_CONCOURS, DATA_PERSONNEL, ...",9524,MAIRIE D'AGNETZ,60007,M14-M14_COM_500_3500,...,None,1,P,None,"[{""Nature"": ""7482"", ""ContNat"": ""74"", ""ArtSpe"":...",21600007500012,21600007500012,VILLE D'AGNETZ,Commune,None


In [27]:
conn = psycopg2.connect(database ="actes_budgetaire", user = "postgres",
                        password = "vGXGLqTY4EF7DCiozYoKlqRRUaC7ECd", host = "ab-postgres",
                        port = "5432")

def get_data(connexion, annexe, year):
    query= f"""SELECT * from annexe 
    JOIN documentbudgetaire 
        ON documentbudgetaire.id = annexe.fk_id_document_budgetaire 
    JOIN collectivite 
        ON collectivite.siret_coll = documentbudgetaire.fk_siret_collectivite 
    WHERE annexe.type_annexe = '{annexe}' 
    AND documentbudgetaire.exercice = '{year}'
    AND documentbudgetaire.nature_dec = '09' 
        LIMIT 100 """
    dat = pd.read_sql_query(query, conn)
    return dat

In [28]:
get_data(connexion=conn, annexe='DATA_APCP', year=2020)

,id,type_annexe,json_annexe,fk_id_document_budgetaire,siret_etablissement,list_annexes,id,libelle,code_insee,nomenclature,...,num_dec,nature_vote,type_budget,id_etabl_princ,json_budget,fk_siret_collectivite,siret_coll,libelle_collectivite,nature_collectivite,departement
0,1152,DATA_APCP,"[{""CodTypAutori"": ""P"", ""CodSTypAutori"": ""S"", ""...",184,24740069000050,"[DATA_EMPRUNT, DATA_TRESORERIE, DATA_TIERS, DA...",184,REGIE ASSAINISSEMENT DU GENEVOIS,74016,M4-M49_D,...,None,1,P,None,"[{""Nature"": ""701241"", ""ArtSpe"": ""false"", ""CodR...",24740069000050,24740069000050,REGIE ASSAINISSEMENT DU GENEVOIS,Communauté de commune à FPU,74
1,1361,DATA_APCP,"[{""CodTypAutori"": ""P"", ""CodSTypAutori"": ""S"", ""...",217,24460048200107,"[DATA_EMPRUNT, DATA_TRESORERIE, DATA_CHARGE, D...",217,MULTISERVICE VIANDE,,M14-M14_COM_SUP3500,...,None,3,A,24460048200123,"[{""Nature"": ""615221"", ""LibCpte"": ""B\u00e2timen...",24460048200107,24460048200107,MULTISERVICE VIANDE,Communes,46
2,1562,DATA_APCP,"[{""CodTypAutori"": ""P"", ""CodSTypAutori"": ""S"", ""...",256,24540060100101,"[DATA_EMPRUNT, DATA_TIERS, DATA_FISCALITE, DAT...",256,CCBP BUDGET PRINCIPAL,54430,M14-M14_COM_SUP3500,...,None,3,P,None,"[{""Nature"": ""001"", ""Fonction"": ""01"", ""ContNat""...",24540060100101,24540060100101,C.C DU BASSIN DE POMPEY,EPCI dont la population est de 3500 habitants ...,None
3,1729,DATA_APCP,"[{""CodTypAutori"": ""P"", ""CodSTypAutori"": ""S"", ""...",285,24770006500029,"[DATA_EMPRUNT, DATA_TIERS, DATA_CONSOLIDATION,...",285,C. de C. PAYS DE L'OURCQ M49 EAU POTABLE ET AS...,77343,M4-M49_D,...,None,1,P,None,"[{""Nature"": ""6061"", ""LibCpte"": ""Fournitures no...",24770006500029,24770006500029,C. de C. PAYS DE L'OURCQ,EPCI,77
4,2008,DATA_APCP,"[{""CodTypAutori"": ""P"", ""CodSTypAutori"": ""S"", ""...",334,20007182700089,"[DATA_EMPRUNT, DATA_CONCOURS, DATA_CONSOLIDATI...",334,BA EAU POTABLE,28905,M4-M49_D,...,None,1,A,20007182700014,"[{""Nature"": ""139111"", ""ContNat"": ""040"", ""CodRD...",20007182700089,20007182700089,CA GRANDANGOULEME,COMMUNE dont la population est de 3500 habitan...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,36796,DATA_APCP,"[{""CodTypAutori"": ""P"", ""CodSTypAutori"": ""S"", ""...",6445,20006711400054,"[DATA_EMPRUNT, DATA_AUTRE_ENGAGEMENT, DATA_CON...",6445,MOBILITE DURABLE,89024,M4-M43_D,...,None,1,A,20006711400013,"[{""Nature"": ""13911"", ""ContNat"": ""040"", ""CodRD""...",20006711400054,20006711400054,COMMUNAUTE DE L'AUXERROIS,,None
96,37642,DATA_APCP,"[{""CodTypAutori"": ""P"", ""CodSTypAutori"": ""S"", ""...",6604,23974001200079,"[DATA_CONCOURS, DATA_CONSOLIDATION, DATA_PATRI...",6604,BUDGET FEDER REUNION,None,M57-M57,...,None,2,P,None,"[{""Nature"": ""65732"", ""Fonction"": ""93052"", ""Art...",23974001200079,23974001200079,BUDGET FEDER REUNION,REGION,None
97,38645,DATA_APCP,"[{""CodTypAutori"": ""P"", ""CodSTypAutori"": ""S"", ""...",6786,21620014700017,"[DATA_EMPRUNT, DATA_TIERS, DATA_CONCOURS, DATA...",6786,COMMUNE AIRE-SUR-LA-LYS,None,M14-M14_COM_SUP3500,...,None,3,P,None,"[{""Nature"": ""6042"", ""LibCpte"": ""Achats de pres...",21620014700017,21620014700017,AIRE SUR LA LYS,,None
98,39982,DATA_APCP,"[{""CodTypAutori"": ""P"", ""CodSTypAutori"": ""S"", ""...",7005,21050046800017,"[DATA_EMPRUNT, DATA_CONCOURS, DATA_RECETTE_AFF...",7005,BUDGET PRINCIPAL,005046,M14-M14_COM_SUP3500,...,None,3,P,None,"[{""Nature"": ""001"", ""Fonction"": ""01"", ""ContNat""...",21050046800017,21050046800017,VILLE D EMBRUN,COMMUNE dont la population est de 3500 habitan...,None


In [ ]:
get_data(connexion=conn,annexe='annexe', year= 'year')

In [3]:
conn = psycopg2.connect(database ="actes_budgetaire", user = "postgres",
                        password = "vGXGLqTY4EF7DCiozYoKlqRRUaC7ECd", host = "ab-postgres", 
                        port = "5432")
query = """
 SELECT * from annexe 
 JOIN documentbudgetaire 
     ON documentbudgetaire.id = annexe.fk_id_document_budgetaire 
 JOIN collectivite 
     ON collectivite.siret_coll = documentbudgetaire.fk_siret_collectivite 
 WHERE annexe.type_annexe = 'DATA_EMPRUNT' 
 AND documentbudgetaire.exercice = '2020'
 AND documentbudgetaire.nature_dec = '09' 
 LIMIT 100
 """
dat = pd.read_sql_query(query, conn)
conn.close()



/home/admin/miniconda3/envs/dataprep/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [5]:
replace_natdec = zip(["01", "02", "03", "09"],
              ["Budget primitif", "Décision modificative", "Budget supplémentaire", "Compte administratif"])
replace_typebud = zip(["P", "A"],
                      ["Principal","Annexe"])
dict_replace = dict({'nature_dec' : dict(replace_natdec),
                     'type_budget' : dict(replace_typebud)})
dict_replace

{'nature_dec': {'01': 'Budget primitif',
  '02': 'Décision modificative',
  '03': 'Budget supplémentaire',
  '09': 'Compte administratif'},
 'type_budget': {'P': 'Principal', 'A': 'Annexe'}}

In [6]:
interesting_columns = ['exercice', 'type_annexe', 'json_annexe','siret_etablissement', 'libelle', 'nomenclature', 'nature_dec', 'nature_vote','type_budget', 'siret_coll', 'libelle_collectivite', 'nature_collectivite']
to_drop = [item for item in list(dat.columns) if item not in interesting_columns]
df = dat.drop(to_drop, axis=1)
df.replace(dict_replace, inplace=True)
df_min = df.head(5)
#[exercice, json_annexe, siret_etablissement, nomenclature
#['exercice', type_annexe', 'json_annexe','siret_etablissement', 'libelle', 'code_insee', 'nomenclature', 'nature_dec', 'num_dec', 'nature_vote','type_budget', 'id_etabl_princ', 'json_budget', 'siret_coll', 'libelle_collectivite', ]

In [7]:
df_min

,type_annexe,json_annexe,siret_etablissement,libelle,nomenclature,exercice,nature_dec,nature_vote,type_budget,siret_coll,libelle_collectivite,nature_collectivite
0,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""C"", ""...",21630291900016,Budget Communal M14,M14-M14_COM_500_3500,2020,Compte administratif,1,Principal,21630291900016,Commune de PUY-GUILLAUME,Commune
1,DATA_EMPRUNT,"[{""CodTypEmpr"": ""09"", ""CodProfilAmort"": ""P"", ""...",21750001600019,BUDGET PRINCIPAL,M57-M57,2020,Compte administratif,2,Principal,21750001600019,Ville de Paris,Commune
2,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""51265.65""}]",21630463400043,BUDGET EAU,M4-M49_D,2020,Compte administratif,1,Annexe,21630463400043,Commune de Viscomtat,Commune
3,DATA_EMPRUNT,"[{""CodTypEmpr"": ""10"", ""RReelFon"": ""50493.40""}]",26390475700014,Budget CCAS 42000,M14-M14_CCAS_INF3500,2020,Compte administratif,1,Principal,26390475700014,Commune de SAINT AMOUR,Commune
4,DATA_EMPRUNT,"[{""CodTypEmpr"": ""01"", ""CodProfilAmort"": ""X"", ""...",21330331600011,74000 BUDGET COMMUNAL M14,M14-M14_COM_INF500,2020,Compte administratif,1,Principal,21330331600011,MAIRIE DE PONDAURAT,Commune


In [12]:
dfs = []
dict_annexe = dict()
for index, row  in df.iterrows():
    json_df_clean = pd.DataFrame(columns=EMPRUNT_FIELD)
    dict_annexe[index] = json.loads(row["json_annexe"])
    for element in dict_annexe[index]:
        if element:
            for field in EMPRUNT_FIELD:
                element.setdefault(field, None)
            
    json_df = pd.json_normalize(dict_annexe[index])
    dfs.append(json_df.assign(**row.drop("json_annexe")))
    #print(json_df)
    #json_df.assign(**row)

In [15]:
df_to_csv = pd.concat(dfs)
df_to_csv.to_csv('./csv/2020-CA-Limit100.csv', index=False)  

In [9]:
EMPRUNT_FIELD = ['AnEncaisse',
  'AnnuitNetDette',
  'Champ_Editeur',
  'CodArticle',
  'CodNatEmpr',
  'CodPeriodRemb',
  'CodPeriodRembDtVote',
  'CodPeriodRembReneg',
  'CodProfilAmort',
  'CodProfilAmortDtVote',
  'CodProfilAmortReneg',
  'CodTypEmpr',
  'CodTypEmprGaranti',
  'CodTypPreteur',
  'CodTypTxCouv',
  'CodTypTxReneg',
  'CodTyptxDtVote',
  'CodTyptxInit',
  'CoutSortie',
  'Couverture',
  'Dt1RembInit',
  'DtDebCouv',
  'DtEmission',
  'DtFinContr',
  'DtFinCouv',
  'DtPeriodeBonif',
  'DtRegltCouv',
  'DtReneg',
  'DtSignInit',
  'DureeAnn',
  'DureeContratInit',
  'DureeContratReneg',
  'DureeRest',
  'DureeRestInit',
  'DureeRestReneg',
  'IndSousJacent',
  'IndSousJacentApresCouv',
  'IndSousJacentAvantCouv',
  'IndSousJacentDtVote',
  'IndexTxVariDtVote',
  'IndexTxVariInit',
  'IndexTxVariReneg',
  'IndiceCouv',
  'IndiceEmpr',
  'IndiceEmprDtVote',
  'LibCpte',
  'LibOrgCoContr',
  'LibOrgaPreteur',
  'MPrimeRecueCouv',
  'MtCRDCouvert',
  'MtCRDRefin',
  'MtCapitalExer',
  'MtCapitalReamenage',
  'MtCapitalRestDu_01_01',
  'MtCapitalRestDu_31_12',
  'MtCharges',
  'MtCommCouv',
  'MtCouv',
  'MtCouvert',
  'MtEmprOrig',
  'MtEmprReneg',
  'MtICNE',
  'MtInt778',
  'MtIntExer',
  'MtPrimePayeeCouv',
  'MtProduits',
  'MtSortie',
  'NatCouv',
  'NomBenefEmprGaranti',
  'NumContrat',
  'NumContratCouv',
  'ObjEmpr',
  'PartGarantie',
  'ProfilAmort',
  'ProfilAmortDtVote',
  'ProvGarantiEmpr',
  'RReelFon',
  'Renegocie',
  'RtAnticipe',
  'Structure',
  'StructureDtVote',
  'StuctureApresCouv',
  'StuctureAvantCouv',
  'Tot1Annuite',
  'TotGarEchoirExer',
  'TxActua',
  'TxActuaInit',
  'TxActuaReneg',
  'TxApresCouv',
  'TxMargeInit',
  'TxMaxi',
  'TxMini',
  'TxPaye',
  'TxRecu',
  'Txinit',
  'TypCouv',
  'TypeSortie']